In [ ]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/mw/work/

In [ ]:
# 查看当前kernel下的package
!pip list --format=columns

In [1]:
# 显示cell运行时长
%load_ext klab-autotime

In [26]:
import pandas as pd
import gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import lightgbm as lgb
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold,StratifiedKFold

In [27]:
seed = 2021

In [28]:
# 读文件
df_train = pd.read_csv('/home/mw/input/tianchi6504/model_sample.csv')
df_train.reset_index()

df_train.head()

,user_id,y,x_001,x_002,x_003,x_004,x_005,x_006,x_007,x_008,...,x_190,x_191,x_192,x_193,x_194,x_195,x_196,x_197,x_198,x_199
0,A00002,0,0.0,32.0,0,0,0,0,0,0,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,A00005,0,0.0,29.0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00006,0,0.0,31.0,0,0,0,0,0,0,...,2.0,2.0,4.0,4.0,6.0,6.0,6.0,6.0,10.0,10.0
3,A00008,0,0.0,22.0,0,0,0,0,0,0,...,3.0,3.0,3.0,3.0,5.0,5.0,3.0,3.0,7.0,7.0
4,A00009,0,0.0,31.0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:

# count_null.sort_values()[::-1]

x_120    10994
x_119    10994
x_118    10994
x_102    10983
x_103    10983
x_104    10983
x_111    10945
x_064    10945
x_062    10944
x_063    10944
x_109    10944
x_110    10944
x_072    10927
x_071    10927
x_073    10927
x_107    10908
x_115    10901
x_116    10901
x_117    10899
x_108    10877
x_106    10877
x_105    10877
x_100    10865
x_101    10865
x_070    10703
x_068    10703
x_069    10703
x_081    10275
x_087    10275
x_086    10275
x_085    10275
x_084    10275
x_083    10275
x_082    10275
x_128    10228
x_130    10228
x_129    10228
x_094    10110
x_092    10110
x_093    10110
x_095    10110
x_114    10103
x_112    10101
x_113    10101
dtype: int64

In [30]:
model_sample.shape

(11017, 199)

In [47]:
no_features = ['y','user_id']  #  删除缺失大于10000的字段
count_null= df_train.isnull().sum() 
count_null = count_null [count_null >10000]
no_features.extend(count_null.index)
no_features 

['y',
 'user_id',
 'x_062',
 'x_063',
 'x_064',
 'x_068',
 'x_069',
 'x_070',
 'x_071',
 'x_072',
 'x_073',
 'x_081',
 'x_082',
 'x_083',
 'x_084',
 'x_085',
 'x_086',
 'x_087',
 'x_092',
 'x_093',
 'x_094',
 'x_095',
 'x_100',
 'x_101',
 'x_102',
 'x_103',
 'x_104',
 'x_105',
 'x_106',
 'x_107',
 'x_108',
 'x_109',
 'x_110',
 'x_111',
 'x_112',
 'x_113',
 'x_114',
 'x_115',
 'x_116',
 'x_117',
 'x_118',
 'x_119',
 'x_120',
 'x_128',
 'x_129',
 'x_130']

In [53]:
for index in count_null.index:  #  趋势值填充
    if index !='x_124' or index !='x_127':
        # print(df_train[index].head())
        df_train[index].fillna(0,inplace=True)
        # print(df_train[index].head())

In [54]:
count_null= df_train.isnull().sum() 
count_null = count_null [count_null >10000]
count_null.head()

Series([], dtype: int64)

In [60]:
# 绘制数据相似度
pd.set_option('display.max_columns',10) 
pd.set_option('display.max_rows',10)
data_train1 = df_train.drop([],axis=1)
train_corr = data_train1.corr()

ax = plt.subplots(figsize=(200,160))
ax = sns.heatmap(train_corr,vmax=.8,square=True,annot=True)


<Figure size 14400x11520 with 2 Axes>

In [63]:
# 查看缺失值
print(f'There are {df_train.isnull().any().sum()} columns in  dataset with missing values.')

There are 117 columns in  dataset with missing values.


In [66]:
have_null_fea_dict = (df_train.isnull().sum()/len(df_train)).to_dict()
fea_null_moreThanHalf = {}
for key,value in have_null_fea_dict.items():
    if value > 0.5:
        fea_null_moreThanHalf[key] = value

In [67]:
fea_null_moreThanHalf


{'x_048': 0.5808296269401834,
 'x_049': 0.5808296269401834,
 'x_050': 0.5808296269401834,
 'x_051': 0.5808296269401834,
 'x_052': 0.5810111645638558,
 'x_053': 0.5810111645638558,
 'x_054': 0.5823726967413997,
 'x_065': 0.8932558772805664,
 'x_066': 0.8987020059907416,
 'x_067': 0.8988835436144141,
 'x_088': 0.6191340655350821,
 'x_089': 0.6191340655350821,
 'x_090': 0.6191340655350821,
 'x_091': 0.6191340655350821,
 'x_096': 0.9051465916311155,
 'x_097': 0.9051465916311155,
 'x_098': 0.9051465916311155,
 'x_099': 0.9051465916311155,
 'x_133': 0.7222474357810656,
 'x_135': 0.7222474357810656,
 'x_136': 0.7222474357810656,
 'x_138': 0.5532359081419624,
 'x_140': 0.5532359081419624,
 'x_141': 0.5532359081419624,
 'x_159': 0.6519016066079695,
 'x_160': 0.6519016066079695,
 'x_161': 0.6519016066079695,
 'x_172': 0.5217391304347826,
 'x_173': 0.5217391304347826,
 'x_174': 0.5217391304347826}

In [70]:
# nan可视化 绘制缺失值超过一半的字段
missing = df_train.isnull().sum()/len(df_train)
missing = missing[missing > 0.5]
missing.sort_values(inplace=True)
missing.plot.bar()

<Figure size 432x288 with 1 Axes>

In [76]:
def get_numerical_serial_fea(data):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    numerical_fea = list(df_train.select_dtypes(exclude=['object']).columns)

    for fea in numerical_fea:
        temp = data[fea].nunique()
        if temp <= 10:
            numerical_noserial_fea.append(fea)
            continue
        numerical_serial_fea.append(fea)
    return numerical_serial_fea,numerical_noserial_fea
numerical_serial_fea,numerical_noserial_fea = get_numerical_serial_fea(df_train)

In [77]:
numerical_serial_fea

['x_002',
 'x_020',
 'x_021',
 'x_026',
 'x_030',
 'x_031',
 'x_034',
 'x_035',
 'x_041',
 'x_043',
 'x_044',
 'x_045',
 'x_046',
 'x_047',
 'x_048',
 'x_050',
 'x_051',
 'x_052',
 'x_053',
 'x_054',
 'x_055',
 'x_057',
 'x_058',
 'x_059',
 'x_060',
 'x_061',
 'x_062',
 'x_064',
 'x_065',
 'x_067',
 'x_068',
 'x_070',
 'x_073',
 'x_074',
 'x_076',
 'x_077',
 'x_078',
 'x_079',
 'x_080',
 'x_081',
 'x_083',
 'x_084',
 'x_085',
 'x_086',
 'x_087',
 'x_088',
 'x_090',
 'x_091',
 'x_092',
 'x_094',
 'x_095',
 'x_096',
 'x_098',
 'x_099',
 'x_104',
 'x_108',
 'x_111',
 'x_112',
 'x_114',
 'x_117',
 'x_120',
 'x_121',
 'x_123',
 'x_124',
 'x_125',
 'x_126',
 'x_127',
 'x_128',
 'x_130',
 'x_131',
 'x_133',
 'x_134',
 'x_135',
 'x_136',
 'x_137',
 'x_138',
 'x_139',
 'x_140',
 'x_141',
 'x_142',
 'x_143',
 'x_144',
 'x_145',
 'x_146',
 'x_147',
 'x_148',
 'x_153',
 'x_154',
 'x_155',
 'x_156',
 'x_157',
 'x_158',
 'x_159',
 'x_160',
 'x_161',
 'x_162',
 'x_163',
 'x_165',
 'x_166',
 'x_167',


In [81]:
#每个数字特征得分布可视化
f = pd.melt(df_train, value_vars=numerical_serial_fea)
g = sns.FacetGrid(f, col="variable",  col_wrap=10, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")

<Figure size 3000x4200 with 132 Axes>

In [80]:
## 用pandas_profiling生成数据报告¶

import pandas_profiling
pfr = pandas_profiling.ProfileReport(df_train)
pfr.to_file("./profiling.html")

In [56]:
# 五折交叉验证
features = [col for col in df_train.columns if col not in no_features]
ycol = 'y'
feature_names = features
model = lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                           num_leaves=64,
                           max_depth=10,
                           learning_rate=0.01,
                           n_estimators=10000,
                           subsample=0.8,
                           feature_fraction=0.6,
                           reg_alpha=10,
                           reg_lambda=12,
                           random_state=seed,
                           is_unbalance=True,
                           metric=['auc','binary_logloss','f1'])


df_oof = []
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names],df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))
    # print(df_oof.loc[val_idx]['prob'])
    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=100,
                          early_stopping_rounds=50)

    pred_val = lgb_model.predict_proba(
        X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
   
    df_oof = df_train.iloc[val_idx][[
        'user_id', ycol]].copy()
    df_oof['pred'] = pred_val
    df_oof.append(df_oof)
    

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model,  X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[46]	train's binary_logloss: 0.446424	train's auc: 0.825713	valid's binary_logloss: 0.457647	valid's auc: 0.775283

Fold_2 Training ================================

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[46]	train's binary_logloss: 0.444653	train's auc: 0.829482	valid's binary_logloss: 0.460067	valid's auc: 0.77309

Fold_3 Training ================================

Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[43]	train's binary_logloss: 0.448288	train's auc: 0.821204	valid's binary_logloss: 0.459251	valid's auc: 0.782449

Fold_4 Training ================================

Training until validation scores don't improve for 50 rounds.
[100]	train's binary_logloss: 0.44469	train's auc: 0.84438	valid's binary_logloss: 0.470156	valid's